# Mode Trajectory Video

In [1]:
import pandas as pd
import pickle
import numpy as np
import itertools
from scipy import stats 
import plotly.graph_objects as go

In [10]:
with open('note_list.pkl', 'rb') as f:
    note_list = pickle.load(f)
with open('note_list_transposed.pkl', 'rb') as f:
    note_list_transposed = pickle.load(f)
with open('measure_list.pkl', 'rb') as f:
    measure_list = pickle.load(f)
with open('files.pkl', 'rb') as f:
    files = pickle.load(f)

In [3]:
def KK(mode, transposition):
    """Krumhansl and Kessler (1982) key profiles for major and minor modes"""
    
    major = [6.20, 2.55, 3.45, 2.85, 4.22, 4.57, 2.67, 5.25, 2.45, 3.35, 2.70, 2.70]
    minor = [6.03, 3.35, 3.67, 5.28, 2.58, 3.55, 2.87, 4.80, 4.35, 2.67, 2.50, 3.42]
    
    major_by_fifths = [None] * len(major)
    minor_by_fifths = [None] * len(minor)
    
    #reorder by fifths
    for i in range(len(major)):
        major_by_fifths[(i*7)%12] = major[i]
        minor_by_fifths[((i*7)+3)%12] = minor[i]
    
    #major = list(map(lambda x: x/sum(major), major))
    #minor = list(map(lambda x: x/sum(minor), minor))
    
    if mode == 0:   
        return major_by_fifths[-transposition:]+major[:-transposition]
    elif mode == 1:
        return minor_by_fifths[-transposition:]+minor[:-transposition]
    else:
        print('0 = major, 1 = minor')

In [4]:
def bag_of_notes(df, tpc='tpc'):
    """Input: DataFrame including the columns ['tpc', 'duration'].
    Output: Note distribution."""
    tpcs = df.tpc
    occurring = np.sort(tpcs.unique())
    bag = pd.DataFrame(index=occurring, columns=['count_a', 'count_n', 'duration_a', 'duration_n'])
    GB = df.groupby('tpc')
    bag.count_a = GB.size()
    bag.count_n = bag.count_a / bag.count_a.sum()
    bag.duration_a = GB['duration'].sum().astype(float)
    bag.duration_n = (bag.duration_a / bag.duration_a.sum()).astype(float)
    if tpc != 'tpc':
        names = tpc.split('+')
        note_names = []
        for n in names:
            if n == 'tpc':
                note_names.append(occurring)
            elif n == 'name':
                note_names.append(tpc2name(occurring))
            elif n == 'degree':
                note_names.append(tpc2rn(occurring))
            elif n == 'pc':
                note_names.append(tpc2pc(occurring))
            else:
                logging.warning("Parameter tpc can only be {'tpc', 'name', 'degree', 'pc'} or a combination such as 'tpc+pc' or 'name+degree+tpc'.")
        L = len(note_names)
        if L == 0:
            note_names.append(bag.index)
            L = 1
        if L == 1:
            bag.index = note_names[0]
        else:
            bag.index = [f"{t[0]} ({', '.join(str(e) for e in t[1:])})" for t in zip(*note_names)]
    return bag

In [5]:
def key_slider(dance, m):
    '''Returns the correlation with major and minor key profiles in a 2-measures window starting at m'''
    window = (note_list_transposed.mc >= m) & (note_list_transposed.mc <= m+2)
    bag = bag_of_notes(note_list_transposed[window].loc[dance], 'tpc')
    
    key_profile = pd.DataFrame(index = list(range(12)), columns = ['profile']).fillna(0)
    for i in bag.index:
        key_profile.loc[i] = bag.duration_n.loc[i]
        
    temp = pd.Series(list(itertools.product([0,1], range(12))), index = pd.MultiIndex.from_product([['major', 'minor'], ['C', 'G',  'D',  'A', 'E','B', 'F#/Gb', 'C#/Db','G#/Ab', 'D#/Eb', 'A#/Bb', 'F']]))
    temp = temp.apply(lambda x: KK(x[0], x[1]-3*x[0]))
    temp = temp.apply(lambda x: stats.pearsonr(key_profile.profile, x)[0])
    
    
    return temp

In [6]:
def key_trajectory(dance):
    local_max = []
    trajectory = pd.DataFrame()
    
    for i in range(len(measure_list.loc[dance])-1):
        keys = key_slider(dance,i).copy()
        trajectory = pd.concat([trajectory, keys], axis = 1, ignore_index = True)
        local_max += [keys.idxmax()]
    
    return [trajectory, set(local_max)]

In [103]:
def key_traj_interactive(dance, speed=0.5, xlabel='Bars', ylabel='Correlation', title='Key Correlations'):    
    """ Plots key trajectories and an animated position marker as vertical line

    Args:
        dance: Index of the dance to plot
        speed: Speed of the marker (bars per animation frame)
        xlabel: Label of the x-axis
        ylabel: Label of the y-axis
        title: Plot title

    """


    key_trajectories = key_trajectory(dance)[0][key_trajectory(dance)[0].index.isin(key_trajectory(dance)[1])].transpose()

    # Define the colors of the trajectories
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
              '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
    maj_colors = ['#1a5276 ', ' #2e86c1 ', ' #7fb3d5 ', ' #154360 ']
    min_colors = ['#FF0010', ' #e74c3c  ', ' #7b241c ', ' #d98880 ']

    n_bars = key_trajectories.shape[0]
    bars = list(range(n_bars))
    data = []

    # Plot the permanent data 
    for idx, key in enumerate(key_trajectories):
        if 'minor' in key:
            color = min_colors[idx%len(min_colors)]
        else:
            color = maj_colors[idx%len(maj_colors)]
        key_graph1=go.Scatter(x=bars, y=key_trajectories[key],
                              mode="lines",
                              showlegend=False,
                              line=dict(color=color))
        key_graph2=go.Scatter(x=bars, y=key_trajectories[key],
                              name=str(key[1])+' '+str(key[0]),
                              mode="lines",
                              line=dict(color=color))
        data.append(key_graph1)
        data.append(key_graph2)

    # Define the buttons
    updatemenus = [dict(buttons=[dict(args=[None, 
                                            dict(frame=dict(duration=500, redraw=False),
                                                 fromcurrent=True,
                                                 transition=dict(duration=3000, easing='quadratic-in-out'))],
                                      label='Play',
                                      method='animate'),
                                 dict(args=[[None],
                                            dict(frame=dict(duration=0, redraw=False),
                                                 mode='immediate',
                                                 transition=dict(duration=0))],
                                          label='Pause',
                                          method='animate')],
                        type='buttons')]

    #Create the moving frames(i.e. the vertical line that moves from left to right)
    bar_axis = np.arange(0, n_bars, speed)

    frames=[go.Frame(
            data=[go.Scatter(
                x=[bar_axis[k], bar_axis[k]],
                y=[-2, 2],
                mode="lines",
                line=dict(color="black", width=2))
            ]) for k in range(len(bar_axis))]

    # Define the layout of the plot
    layout=go.Layout(width=950, height=600,
                     xaxis=dict(range=[0, max(bars)], autorange=False, zeroline=False),
                     yaxis=dict(range=[-0.4, 1], autorange=False, zeroline=False),
                     xaxis_title=xlabel,
                     yaxis_title=ylabel,
                     title=title,
                     hovermode="closest",
                     updatemenus=updatemenus)

    # Create figure
    fig = go.Figure(data, layout, frames)
    fig.show()

In [104]:
key_traj_interactive(24, title="Key trajectories for Trio D41 Nr.12") 

## Static plot of key trajectories with dropdown

In [100]:
import cufflinks as cf

In [111]:
# Specifiy the dance IDs that should appear in the dropdown
selection = [1, 2, 3, 24]

buttons = []
lines = []
lines_idx_dict = {}
counter = 0

# Create the lines for all trajectories
for idx, dance in enumerate(selection):
    counter_before = counter
    key_trajectories = key_trajectory(dance)[0][key_trajectory(dance)[0].index.isin(key_trajectory(dance)[1])].transpose()
    if idx == 0:
        visible=True
    else:
        visible=False
        
    key_string = str(key[1])+' '+str(key[0])
    
    for key in key_trajectories:
        line = go.Scatter(x=np.arange(len(key_trajectories_list[0])),
                          y=key_trajectories[key].to_list(),
                          name=key_string,
                          visible=visible,
                          showlegend=True)
        
        lines.append(line)
        counter = counter + 1
        
    lines_idx_dict[dance] = (counter_before, counter)
    
# Create the buttons that toggle the visibility
for idx, dance in enumerate(selection):
    true_false_list = [False] * len(lines)
    label = 'D' + str(files.loc[dance]['D']) + ' ' + str(files.loc[dance]['dance']) + ' N. ' + str(files.loc[dance]['no'])    
    visible_list = true_false_list
    
    for i in range(lines_idx_dict[dance][0], lines_idx_dict[dance][1]):
        true_false_list[i] = True
    
    button = dict(label=label,
                  method='update',
                  args=[{'visible': true_false_list}])
    buttons.append(button)
    
updatemenus = [dict(buttons=buttons, direction='down', showactive=True)]

layout = go.Layout(
    updatemenus=updatemenus,
)

figure = go.Figure(data=lines, layout=layout)
po.iplot(figure)

C major
